# Facility Location Optimization

Facility Location Optimization is a important area of operations research and supply chain management that focuses on finding the most cost-effective locations for operating facilities like warehouses, factories, or distribution centers. This optimization considers various factors, including the proximity to suppliers and customers, transportation costs, and the fixed costs associated with opening and operating each facility.

The goal of Facility Location Optimization is to minimize the overall costs while satisfying service level requirements and demand at different customer points. This involves a careful balance between minimizing fixed costs (such as the cost of opening a facility) and variable costs (such as transportation costs from the facility to the customers).

Mathematically, this problem is often modeled using binary decision variables to represent the opening of facilities and continuous variables to represent the flow of goods from facilities to demand points. The objective function aims to minimize the sum of fixed and variable costs, subject to constraints ensuring that demand is met and that goods can only be shipped from open facilities.

This optimization problem is a cornerstone in strategic planning for logistics and distribution networks, enabling companies to efficiently allocate resources, reduce costs, and improve service levels.

## Scenario

In our project, we're tackling a scenario where a logistics company needs to decide on the best locations for its new distribution centers. The company aims to serve a set of cities, each with its own demand for goods. The challenge is to figure out where to place these distribution centers to minimize costs, which include the cost of opening the centers and the cost of shipping goods to the cities.

We have a list of potential locations for these centers and each location has a fixed cost to open. Also, shipping costs vary depending on how far a city is from a distribution center. Our job is to find the optimal mix of which centers to open and how much to ship from each center to meet all the cities' demands at the lowest possible cost.

This scenario is a classic example of facility location optimization, where we need to make decisions based on various costs and constraints. We'll use mathematical modeling to represent this problem and then solve it using a powerful optimization solver.

### Mathematical Model for Facility Location Optimization

#### Sets and Indices
$$
\begin{align*}
&I: \text{Set of potential locations for distribution centers, indexed by } i. \\
&J: \text{Set of demand points (cities), indexed by } j.
\end{align*}
$$

#### Parameters
$$
\begin{align*}
&F_i: \text{Fixed cost of opening a distribution center at location } i. \\
&C_{ij}: \text{Transportation cost per unit from distribution center } i \text{ to demand point } j. \\
&D_j: \text{Demand at demand point } j.
\end{align*}
$$

#### Decision Variables
$$
\begin{align*}
&x_{ij}: \text{Quantity of goods transported from distribution center } i \text{ to demand point } j. \\
&y_i: \text{Binary variable that equals 1 if a distribution center is opened at location } i, \text{ and 0 otherwise.}
\end{align*}
$$

#### Objective Function
$$
\text{Minimize} \quad Z = \sum_{i \in I} F_i y_i + \sum_{i \in I} \sum_{j \in J} C_{ij} x_{ij}
$$
The objective function aims to minimize the total cost, which includes the fixed costs of opening distribution centers and the variable transportation costs.

#### Constraints

1. **Demand Satisfaction**: Each demand point's demand must be met by the distribution centers.
$$
\sum_{i \in I} x_{ij} \geq D_j, \quad \forall j \in J
$$

2. **Supply Linking**: A distribution center can only supply goods if it is open.
$$
x_{ij} \leq D_j y_i, \quad \forall i \in I, \forall j \in J
$$

3. **Binary and Non-Negativity Constraints**: The decision variables must adhere to binary and non-negativity restrictions.
$$
y_i \in \{0, 1\}, \quad \forall i \in I
$$
$$
x_{ij} \geq 0, \quad \forall i \in I, \forall j \in J
$$


In [1]:
import gurobipy as gp
from gurobipy import GRB


N = 5  # Number of potential locations
M = 3  # Number of demand points
F = [1000, 1200, 1100, 1300, 1250]  # Fixed costs
C = [[20, 24, 22], [25, 27, 23], [24, 26, 21], [23, 22, 25], [22, 20, 24]]  # Transportation costs
D = [500, 600, 700]  # Demands

# Model
model = gp.Model("FacilityLocation")

# Variables
x = model.addVars(N, M, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(N, vtype=GRB.BINARY, name="y")

# Objective function
model.setObjective(gp.quicksum(F[i] * y[i] for i in range(N)) +
                   gp.quicksum(C[i][j] * x[i, j] for i in range(N) for j in range(M)), GRB.MINIMIZE)

# Constraints
for j in range(M):
    model.addConstr(gp.quicksum(x[i, j] for i in range(N)) >= D[j], f"Demand_{j}")

for i in range(N):
    for j in range(M):
        model.addConstr(x[i, j] <= D[j] * y[i], f"Link_{i}_{j}")

# Solve
model.optimize()

# Output
if model.status == GRB.OPTIMAL:
    print(f"Optimal Objective: {model.ObjVal}")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}")

    # Generate report
    model.write("optimization_report.sol")

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18 rows, 20 columns and 45 nonzeros
Model fingerprint: 0x6f1763f4
Variable types: 15 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 7e+02]
Presolve time: 0.00s
Presolved: 18 rows, 20 columns, 45 nonzeros
Variable types: 0 continuous, 20 integer (5 binary)
Found heuristic solution: objective 40050.000000

Root relaxation: objective 3.965000e+04, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    39650.00000